In [22]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [23]:
FILE_PATH = r"D:\aistudy\metacode_study\06-DocumentLoader\data\KDS140000Structure.pdf"
print(FILE_PATH)

D:\aistudy\metacode_study\06-DocumentLoader\data\KDS140000Structure.pdf


In [25]:
"""
Nougat에 PDF를 직접 전달하여 compression_type 오류 우회
이미지 변환 없이 PDF에서 바로 LaTeX 추출
"""

import torch
from typing import List, Dict, Optional, Generator
from dataclasses import dataclass
import os


@dataclass
class Document:
    """RAG에서 사용할 문서 객체"""
    page_content: str
    metadata: Dict
    
    def __repr__(self):
        preview = self.page_content[:100].replace('\n', ' ')
        return f"Document(page={self.metadata.get('page')}, content='{preview}...')"


class NougatPDFLoader:
    """Nougat으로 PDF를 직접 처리하는 로더 (이미지 변환 불필요)"""
    
    def __init__(
        self,
        model_name: str = "facebook/nougat-base",
        use_gpu: bool = True,
        batch_size: int = 1
    ):
        """
        Args:
            model_name: Nougat 모델 이름
            use_gpu: GPU 사용 여부
            batch_size: 배치 크기 (메모리에 따라 조정)
        """
        self.model_name = model_name
        self.use_gpu = use_gpu and torch.cuda.is_available()
        self.batch_size = batch_size
        self.model = None
        self.device = "cuda" if self.use_gpu else "cpu"
    
    def load_model(self):
        """Nougat 모델 로드"""
        if self.model is not None:
            return
        
        print(f"Nougat 모델 로딩 중... (device: {self.device})")
        
        try:
            from nougat import NougatModel
            from nougat.utils.checkpoint import get_checkpoint
            
            # 모델 로드
            self.model = NougatModel.from_pretrained(self.model_name)
            self.model = self.model.to(self.device)
            self.model.eval()
            
            print("✓ 모델 로딩 완료")
        
        except ImportError:
            raise ImportError(
                "Nougat이 설치되지 않았습니다.\n"
                "설치: pip install nougat-ocr"
            )
    
    def process_pdf_direct(
        self, 
        pdf_path: str, 
        start_page: int = 0,
        end_page: Optional[int] = None
    ) -> List[str]:
        """
        PDF를 직접 처리 (이미지 변환 없이)
        
        Args:
            pdf_path: PDF 파일 경로
            start_page: 시작 페이지 (0부터 시작)
            end_page: 끝 페이지 (None이면 끝까지)
        
        Returns:
            페이지별 텍스트 리스트
        """
        from nougat.dataset.rasterize import rasterize_paper
        from nougat.utils.dataset import LazyDataset
        from nougat.postprocessing import markdown_compatible
        from torch.utils.data import DataLoader
        
        if self.model is None:
            self.load_model()
        
        print(f"\nPDF 처리 시작: {os.path.basename(pdf_path)}")
        
        # PDF를 Nougat Dataset으로 변환
        try:
            # rasterize_paper로 PDF를 이미지 리스트로 변환
            images = rasterize_paper(
                pdf_path,
                return_pil=True,
                pages=list(range(start_page, end_page if end_page else 9999))
            )
            
            print(f"총 {len(images)}개 페이지 발견")
            
            # Dataset 생성
            dataset = LazyDataset(
                images,
                partial(self.model.encoder.prepare_input, random_padding=False),
            )
            
            # DataLoader 생성
            dataloader = DataLoader(
                dataset,
                batch_size=self.batch_size,
                shuffle=False,
                collate_fn=LazyDataset.ignore_none_collate
            )
            
            # 추론 수행
            predictions = []
            
            with torch.no_grad():
                for idx, (sample, is_last_page) in enumerate(dataloader):
                    if sample is None:
                        continue
                    
                    # GPU로 이동
                    if self.use_gpu:
                        sample = sample.to(self.device)
                    
                    # 모델 추론
                    model_output = self.model.inference(
                        image_tensors=sample,
                        early_stopping=True
                    )
                    
                    # 후처리
                    for output in model_output["predictions"]:
                        output = markdown_compatible(output)
                        predictions.append(output)
                    
                    print(f"  페이지 {idx + 1}/{len(images)} 완료")
            
            return predictions
        
        except Exception as e:
            print(f"오류 발생: {e}")
            import traceback
            traceback.print_exc()
            return []
    
    def load_single_page(self, pdf_path: str, page_num: int) -> Document:
        """
        단일 페이지 로드 (1부터 시작)
        
        Args:
            pdf_path: PDF 파일 경로
            page_num: 페이지 번호 (1부터 시작)
        
        Returns:
            Document 객체
        """
        import time
        
        start_time = time.time()
        
        # 페이지 인덱스는 0부터
        results = self.process_pdf_direct(
            pdf_path,
            start_page=page_num - 1,
            end_page=page_num
        )
        
        processing_time = time.time() - start_time
        
        if not results:
            raise RuntimeError(f"페이지 {page_num} 처리 실패")
        
        return Document(
            page_content=results[0],
            metadata={
                'source': pdf_path,
                'page': page_num,
                'processing_time': processing_time
            }
        )
    
    def load_pages(
        self,
        pdf_path: str,
        page_range: Optional[tuple] = None
    ) -> Generator[Document, None, None]:
        """
        여러 페이지를 스트리밍 방식으로 로드
        
        Args:
            pdf_path: PDF 파일 경로
            page_range: (시작, 끝) 페이지 번호 (1부터 시작)
        
        Yields:
            Document 객체
        """
        import fitz
        
        # 전체 페이지 수 확인
        doc = fitz.open(pdf_path)
        total_pages = len(doc)
        doc.close()
        
        if page_range is None:
            start, end = 1, total_pages
        else:
            start, end = page_range
            end = min(end, total_pages)
        
        print(f"총 {total_pages}페이지 중 {start}~{end} 페이지 처리")
        
        # 모델 로드
        self.load_model()
        
        # 페이지 범위 처리 (0-based)
        results = self.process_pdf_direct(
            pdf_path,
            start_page=start - 1,
            end_page=end
        )
        
        # Document 객체로 변환
        for idx, text in enumerate(results):
            page_num = start + idx
            yield Document(
                page_content=text,
                metadata={
                    'source': pdf_path,
                    'page': page_num,
                    'total_pages': total_pages
                }
            )
    
    def load_all(
        self,
        pdf_path: str,
        page_range: Optional[tuple] = None
    ) -> List[Document]:
        """
        모든 페이지를 한 번에 로드
        
        Args:
            pdf_path: PDF 파일 경로
            page_range: (시작, 끝) 페이지 번호
        
        Returns:
            Document 객체 리스트
        """
        return list(self.load_pages(pdf_path, page_range))
    
    def unload_model(self):
        """모델 언로드"""
        if self.model is not None:
            del self.model
            self.model = None
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            print("✓ 모델 언로드 완료")


# ============================================================
# 사용 예제
# ============================================================

def example_single_page_direct(pdf_path: str, page_num: int = 1, use_gpu: bool = True):
    """
    단일 페이지 처리 (PDF 직접 전달)
    
    Args:
        pdf_path: PDF 파일 경로
        page_num: 페이지 번호 (1부터 시작)
        use_gpu: GPU 사용 여부
    
    Returns:
        Document 객체
    """
    print("\n" + "="*60)
    print("Nougat PDF 직접 처리 (이미지 변환 없음)")
    print("="*60 + "\n")
    
    # 로더 초기화
    loader = NougatPDFLoader(use_gpu=use_gpu)
    
    # 페이지 로드
    try:
        document = loader.load_single_page(pdf_path, page_num)
        
        print(f"\n문서 정보:")
        print(f"  페이지: {document.metadata['page']}")
        print(f"  처리시간: {document.metadata['processing_time']:.2f}초")
        print(f"\n내용 미리보기:")
        print(document.page_content[:500])
        
        return document
    
    finally:
        loader.unload_model()


def example_multiple_pages_direct(pdf_path: str, page_range: tuple = (1, 5), use_gpu: bool = True):
    """
    여러 페이지 처리 (PDF 직접 전달)
    
    Args:
        pdf_path: PDF 파일 경로
        page_range: (시작, 끝) 페이지 번호
        use_gpu: GPU 사용 여부
    
    Returns:
        Document 객체 리스트
    """
    print("\n" + "="*60)
    print("Nougat PDF 다중 페이지 처리")
    print("="*60 + "\n")
    
    # 로더 초기화
    loader = NougatPDFLoader(use_gpu=use_gpu, batch_size=1)
    
    try:
        documents = []
        
        for doc in loader.load_pages(pdf_path, page_range):
            print(f"\n페이지 {doc.metadata['page']} 완료")
            print(f"  내용 길이: {len(doc.page_content)} 문자")
            documents.append(doc)
            
            # 파일로 저장
            with open(f"page_{doc.metadata['page']}.md", 'w', encoding='utf-8') as f:
                f.write(doc.page_content)
        
        print(f"\n총 {len(documents)}개 문서 처리 완료")
        return documents
    
    finally:
        loader.unload_model()


# 필요한 import 추가
from functools import partial



In [26]:
doc = example_single_page(FILE_PATH, page_num=34, dpi=300, use_gpu=True)


예제 1: 단일 페이지 처리

페이지 34 OCR 처리 중...
  CLI 오류, API 모드로 전환: Nougat CLI 오류: D:\aistudy\metacode_study\tenv\Lib\site-packages\nougat\transforms.py:108: UserWarning: Argument(s) 'always_apply' are not valid for transform BasicTransform
  super().__init__(always_apply=always_apply, p=p)
D:\aistudy\metacode_study\tenv\Lib\site-packages\nougat\transforms.py:42: UserWarning: Argument(s) 'always_apply' are not valid for transform BasicTransform
  super().__init__(always_apply=always_apply, p=p)
D:\aistudy\metacode_study\tenv\Lib\site-packages\nougat\transforms.py:76: UserWarning: Argument(s) 'always_apply' are not valid for transform BasicTransform
  super().__init__(always_apply=always_apply, p=p)
D:\aistudy\metacode_study\tenv\Lib\site-packages\nougat\transforms.py:123: UserWarning: Argument(s) 'cval' are not valid for transform Affine
  alb.Affine(shear={"x": (0, 3), "y": (-3, 0)}, cval=(255, 255, 255), p=0.03),
D:\aistudy\metacode_study\tenv\Lib\site-packages\albumentations\core\validation.

d:\aistudy\metacode_study\tenv\Lib\site-packages\nougat\transforms.py:108: UserWarning: Argument(s) 'always_apply' are not valid for transform BasicTransform
  super().__init__(always_apply=always_apply, p=p)
d:\aistudy\metacode_study\tenv\Lib\site-packages\nougat\transforms.py:42: UserWarning: Argument(s) 'always_apply' are not valid for transform BasicTransform
  super().__init__(always_apply=always_apply, p=p)
d:\aistudy\metacode_study\tenv\Lib\site-packages\nougat\transforms.py:76: UserWarning: Argument(s) 'always_apply' are not valid for transform BasicTransform
  super().__init__(always_apply=always_apply, p=p)
d:\aistudy\metacode_study\tenv\Lib\site-packages\nougat\transforms.py:123: UserWarning: Argument(s) 'cval' are not valid for transform Affine
  alb.Affine(shear={"x": (0, 3), "y": (-3, 0)}, cval=(255, 255, 255), p=0.03),
d:\aistudy\metacode_study\tenv\Lib\site-packages\nougat\transforms.py:124: UserWarning: Argument(s) 'value' are not valid for transform ShiftScaleRotate
 

ValueError: 1 validation error for InitSchema
compression_type
  Input should be 'jpeg' or 'webp' [type=literal_error, input_value=95, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error